In [1]:
dbutils.fs.put("dbfs:/databricks/init/test/tensorflow.sh","""#!/bin/bash

set -e

/databricks/python/bin/python -V
. /databricks/conda/etc/profile.d/conda.sh
conda install -y conda=4.6
conda activate /databricks/python

conda install -y tensorflow-gpu=2.0 setuptools=41
""")

Wrote 209 bytes.
 Out[ 6 ]: True

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder, scale
from glob import glob

In [3]:
#test if GPU is available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

--------------------------------------------------------------------------- 
 SystemError Traceback (most recent call last)
 <command-1277933326543323> in <module> 
 2 device_name = tf . test . gpu_device_name ( ) 
 3 if device_name != '/device:GPU:0' : 
 ----> 4 raise SystemError ( 'GPU device not found' ) 
 5 print ( 'Found GPU at: {}' . format ( device_name ) ) 

 SystemError : GPU device not found

In [4]:
#benchmark to confirm GPU is being utilized
import time

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

def timer(meth,n):
  start = time.time()
  for i in range(n):
    globals()[meth]()
  end = time.time()
  return end - start

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timer('cpu',10)
print(cpu_time)
print('GPU (s):')
gpu_time = timer('gpu',10)
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.91989088058
GPU (s):
0.211874961853
GPU speedup over CPU: 13x

In [6]:
data_path = "dbfs:/FileStore/tmp/wisdm"
data_path_local = "/{}".format(data_path.replace(":",""))

In [7]:
df_act =pd.read_csv(data_path_local+"/activity_key.txt",names=['ActivityDescription','ActivityCode'],sep=' = ')

read_data = lambda x: pd.read_csv(x,names=['Subject-id','ActivityCode','Timestamp','x','y','z'])

df_gyro=pd.concat(map(read_data,glob(data_path_local+"/watch/gyro/*"))).dropna()
df_accel=pd.concat(map(read_data,glob(data_path_local+"/watch/accel/*"))).dropna()

/local_disk0/tmp/1576012815558-0/PythonShell.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
 from __future__ import absolute_import

In [8]:
#it's a good practice to scale features for NN since value swings can throw off gradients during training
for col in 'xyz':
  df_gyro[col] = scale(df_gyro[col])
  df_accel[col] = scale(df_accel[col])

In [9]:
#We will feed the model sequences of specified lenght. 
#This variable specifies the sequence length in seconds
seq_length = 10

In [10]:
#we need to prepare the data so that it's shape is in the form of (samples, time steps, features)
gx_train =[]
gy_train = []
gx_test = []
gy_test = []
for name, group in df_gyro.sort_values(by=['Subject-id','ActivityCode']).groupby(['Subject-id','ActivityCode']):
  group = group.sort_values(by="Timestamp")
  #get sequences of specified length
  seqs = np.split(group[['x','y','z']].values,np.arange(seq_length * 20, 3600, seq_length * 20))
  if name[0] <= 1635:
    gx_train += seqs
    gy_train += ([name[1]] * len(seqs))
  else:
    gx_test += seqs
    gy_test += ([name[1]] * len(seqs))

gx_train = np.stack(pad_sequences(gx_train,dtype='float',maxlen=seq_length * 20))
gx_test = np.stack(pad_sequences(gx_test,dtype='float',maxlen=seq_length * 20))
gy_train = np.asarray(gy_train).reshape(-1,1)
gy_test = np.asarray(gy_test).reshape(-1,1)


In [11]:
ax_train =[]
ay_train = []
ax_test = []
ay_test = []
for name, group in df_accel.sort_values(by=['Subject-id','ActivityCode']).groupby(['Subject-id','ActivityCode']):
  group = group.sort_values(by="Timestamp")
  #get sequences of specified length
  seqs = np.split(group[['x','y','z']].values,np.arange(seq_length * 20, 3600, seq_length * 20))
  if name[0] <= 1635:
    ax_train += seqs
    ay_train += ([name[1]] * len(seqs))
  else:
    ax_test += seqs
    ay_test += ([name[1]] * len(seqs))

ax_train = np.stack(pad_sequences(gx_train,dtype='float',maxlen=seq_length * 20))
ax_test = np.stack(pad_sequences(gx_test,dtype='float',maxlen=seq_length * 20))
ay_train = np.asarray(gy_train).reshape(-1,1)
ay_test = np.asarray(gy_test).reshape(-1,1)

In [12]:
#categorical targets must be one hot encoded
ohe = OneHotEncoder()
ohe.fit(df_gyro.ActivityCode.unique().reshape(-1,1))

Out[11]: OneHotEncoder(categorical_features=None, categories=None,
 dtype=<class 'numpy.float64'>, handle_unknown='error',
 n_values=None, sparse=True)

In [13]:
g_input = Input(shape=(seq_length * 20,3),dtype='float',name='gyro_input')
a_input = Input(shape=(seq_length * 20,3),dtype='float',name='accel_input')
g_conv = layers.Conv1D(64,kernel_size=20,padding='same', activation='relu')(g_input)
g_max = layers.MaxPool1D(2,padding='same')(g_conv)
a_conv = layers.Conv1D(64,kernel_size=20,padding='same', activation='relu')(a_input)
a_max = layers.MaxPool1D(2,padding='same')(a_conv)
g_gru = layers.GRU(64,dropout=0.4, return_sequences=True)(g_max)
a_gru = layers.GRU(64, dropout=0.4, return_sequences=True)(a_max)
conc = layers.concatenate([g_gru,a_gru], axis=-1)
dense = layers.Dense(64,activation="relu")(conc)
flat = layers.Flatten()(dense)
final = layers.Dense(18,activation='softmax',name="prediction")(flat)

model = Model([g_input,a_input],final)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=["acc"])


WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/nn_ops.py:61: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 elif not isinstance(value, collections.Sized):

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type) Output Shape Param # Connected to 
==================================================================================================
gyro_input (InputLayer) [(None, 200, 3)] 0 
__________________________________________________________________________________________________
accel_input (InputLayer) [(None, 200, 3)] 0 
__________________________________________________________________________________________________
conv1d (Conv1D) (None, 200, 64) 3904 gyro_input[0][0] 
__________________________________________________________________________________________________
conv1d_1 (Conv1D) (None, 200, 64) 3904 accel_input[0][0] 
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 100, 64) 0 conv1d[0][0] 
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D) (None, 100, 64) 0 conv1d_1[0][0] 
__________________________________________________________________________________________________
gru (GRU) (None, 100, 64) 24768 max_pooling1d[0][0] 
__________________________________________________________________________________________________
gru_1 (GRU) (None, 100, 64) 24768 max_pooling1d_1[0][0] 
__________________________________________________________________________________________________
concatenate (Concatenate) (None, 100, 128) 0 gru[0][0] 
 gru_1[0][0] 
__________________________________________________________________________________________________
dense (Dense) (None, 100, 64) 8256 concatenate[0][0] 
__________________________________________________________________________________________________
flatten (Flatten) (None, 6400) 0 dense[0][0] 
__________________________________________________________________________________________________
prediction (Dense) (None, 18) 115218 flatten[0][0] 
==================================================================================================
Total params: 180,818
Trainable params: 180,818
Non-trainable params: 0
__________________________________________________________________________________________________

In [15]:
model.fit([gx_train,ax_train], ohe.transform(gy_train).todense(), batch_size = 32, epochs=10)

Train on 11628 samples
Epoch 1/10
 32/11628 [..............................] - ETA: 25:21 - loss: 2.8775 - acc: 0.0312 96/11628 [..............................] - ETA: 8:34 - loss: 2.9938 - acc: 0.0417  160/11628 [..............................] - ETA: 5:12 - loss: 2.9028 - acc: 0.0625 224/11628 [..............................] - ETA: 3:46 - loss: 2.8203 - acc: 0.0670 288/11628 [..............................] - ETA: 2:58 - loss: 2.8202 - acc: 0.0764 352/11628 [..............................] - ETA: 2:27 - loss: 2.7711 - acc: 0.0739 416/11628 [>.............................] - ETA: 2:06 - loss: 2.7516 - acc: 0.0745 480/11628 [>.............................] - ETA: 1:51 - loss: 2.7082 - acc: 0.0938 544/11628 [>.............................] - ETA: 1:39 - loss: 2.6635 - acc: 0.1029 608/11628 [>.............................] - ETA: 1:29 - loss: 2.6223 - acc: 0.1102 672/11628 [>.............................] - ETA: 1:21 - loss: 2.5829 - acc: 0.1176 736/11628 [>.............................] - ETA: 1:15 - loss: 2.5594 - acc: 0.1250 800/11628 [=>............................] - ETA: 1:10 - loss: 2.5390 - acc: 0.1388 864/11628 [=>............................] - ETA: 1:05 - loss: 2.4998 - acc: 0.1470 928/11628 [=>............................] - ETA: 1:01 - loss: 2.4535 - acc: 0.1659 992/11628 [=>............................] - ETA: 58s - loss: 2.4247 - acc: 0.1714  1056/11628 [=>............................] - ETA: 55s - loss: 2.3884 - acc: 0.1837 1120/11628 [=>............................] - ETA: 52s - loss: 2.3606 - acc: 0.1911 1184/11628 [==>...........................] - ETA: 50s - loss: 2.3330 - acc: 0.1976 1248/11628 [==>...........................] - ETA: 47s - loss: 2.2977 - acc: 0.2083 1312/11628 [==>...........................] - ETA: 46s - loss: 2.2761 - acc: 0.2111 1376/11628 [==>...........................] - ETA: 44s - loss: 2.2444 - acc: 0.2209 1440/11628 [==>...........................] - ETA: 42s - loss: 2.2193 - acc: 0.2285 1504/11628 [==>...........................] - ETA: 41s - loss

In [16]:
model.evaluate([gx_test,ax_test],ohe.transform(gy_test).todense(),batch_size=32)

4698/1 [========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [17]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
pred = np.argmax(model.predict([gx_test,ax_test]),axis=1)
act = np.argmax(ohe.transform(gy_test),axis=1)

In [19]:
mat = pd.DataFrame(confusion_matrix(act,pred),columns=df_act['ActivityDescription'].unique(),
                   index=df_act['ActivityDescription'].unique())

In [20]:
plt.figure(figsize = (12,7))
sns.heatmap(mat, cmap="Reds",annot=True, annot_kws={"size": 10}, fmt='d', linewidths=.5)
sns.despine()
plt.tight_layout()
display()